# Notebook para fazer ETL do csv CC GENERAL para um banco de dados MySQL

## Bibliotecas

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import mysql.connector
from mysql.connector import errorcode
from mysql.connector import connect

In [2]:
path_dotenv = r'C:\Users\erico\Documents\projeto-clusterizacao\customer-segmentation\env\.env'
load_dotenv(path_dotenv)

True

In [3]:
# ler cvs
df = pd.read_csv(r'C:\Users\erico\Documents\projeto-clusterizacao\customer-segmentation\data\raw\CC GENERAL.csv')

In [4]:
df.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [5]:
# Identificar os tipos de dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           8950 non-null   object 
 1   BALANCE                           8950 non-null   float64
 2   BALANCE_FREQUENCY                 8950 non-null   float64
 3   PURCHASES                         8950 non-null   float64
 4   ONEOFF_PURCHASES                  8950 non-null   float64
 5   INSTALLMENTS_PURCHASES            8950 non-null   float64
 6   CASH_ADVANCE                      8950 non-null   float64
 7   PURCHASES_FREQUENCY               8950 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY        8950 non-null   float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  8950 non-null   float64
 10  CASH_ADVANCE_FREQUENCY            8950 non-null   float64
 11  CASH_ADVANCE_TRX                  8950 non-null   int64  
 12  PURCHA

In [6]:
# Estabelecer a conexão com o banco de dados MySQL com SSL
conn = connect(
    user='2og9lykdi1kfw9a9g7r2',
    password=os.environ.get("MYSQL_PASSWORD"),
    host='aws.connect.psdb.cloud',
    database='projeto_clusterizacao',
    ssl_ca='../cacert-2023-01-10.pem',
    use_pure=True
)

In [7]:
# Verificar se a tabela existe
cursor = conn.cursor()
table_name = 'customer_credit_card'
check_table_query = f"SHOW TABLES LIKE '{table_name}'"
cursor.execute(check_table_query)

if cursor.fetchone():
    print(f"A tabela {table_name} existe.")
else:
    create_table_query = '''
        CREATE TABLE customer_credit_card (
            CUST_ID VARCHAR(10),
            BALANCE FLOAT(10, 2),
            BALANCE_FREQUENCY FLOAT(10, 2),
            PURCHASES FLOAT(10, 2),
            ONEOFF_PURCHASES FLOAT(10, 2),
            INSTALLMENTS_PURCHASES FLOAT(10, 2),
            CASH_ADVANCE FLOAT(10, 2),
            PURCHASES_FREQUENCY FLOAT(10, 2),
            ONEOFF_PURCHASES_FREQUENCY FLOAT(10, 2),
            PURCHASES_INSTALLMENTS_FREQUENCY FLOAT(10, 2),
            CASH_ADVANCE_FREQUENCY FLOAT(10, 2),
            CASH_ADVANCE_TRX INT,
            PURCHASES_TRX INT,
            CREDIT_LIMIT FLOAT(10, 2),
            PAYMENTS FLOAT(10, 2),
            MINIMUM_PAYMENTS FLOAT(10, 2),
            PRC_FULL_PAYMENT FLOAT(10, 2),
            TENURE INT
        )
    '''
    cursor.execute(create_table_query)
    print(f"A tabela {table_name} foi criada.")

A tabela customer_credit_card foi criada.


In [9]:
# Verificar se a tabela foi criada ou já existia
cursor.execute(check_table_query)

if cursor.fetchone():
    # Executar a operação TRUNCATE para remover os dados existentes
    truncate_query = "TRUNCATE TABLE customer_credit_card"
    cursor.execute(truncate_query)
    print("Dados existentes removidos.")

    # Preparar a instrução INSERT INTO
    insert_query = '''
        INSERT INTO customer_credit_card (
            CUST_ID, BALANCE, BALANCE_FREQUENCY, PURCHASES, ONEOFF_PURCHASES,
            INSTALLMENTS_PURCHASES, CASH_ADVANCE, PURCHASES_FREQUENCY,
            ONEOFF_PURCHASES_FREQUENCY, PURCHASES_INSTALLMENTS_FREQUENCY,
            CASH_ADVANCE_FREQUENCY, CASH_ADVANCE_TRX, PURCHASES_TRX,
            CREDIT_LIMIT, PAYMENTS, MINIMUM_PAYMENTS, PRC_FULL_PAYMENT, TENURE
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''

    # Executar a instrução INSERT INTO com todos os valores
    values = [tuple(row) for row in df.itertuples(index=False)]
    cursor.executemany(insert_query, values)

    # Certificar-se de que os dados sejam confirmados no banco de dados
    conn.commit()

    # Fechar o cursor e a conexão
    cursor.close()
    conn.close()

Dados existentes removidos.
